# 💙 <span style="color:navy;text-shadow: 0px 2px 3px gray;">ihaskell-widgets</span> Revival Celebration Notebook 💙

This demonstration notebook is to celebrate the successful completion of [David Davó Laviña’s Google Summer of Code 2021 project](https://summerofcode.withgoogle.com/projects/#5497023849037824) to revive [*ihaskell-widgets*](https://github.com/gibiansky/IHaskell/tree/master/ihaskell-display/ihaskell-widgets). 🎉

Here is [David Davó Laviña’s report on __Fixing IHaskell-Widgets__](https://gsoc21.ddavo.me/).

For David’s *ihaskell-widgets* example notebooks and tutorials, see the `/ihaskell_examples/ihaskell-widgets` directory.

For David’s other work, see [ddavo.me English](https://ddavo.me/en), [ddavo.me Español](https://ddavo.me/es), [ddavo.me 日本語](https://ddavo.me/jp).

Many experienced Haskell programmers tried and failed to revive the *ihaskell-widgets* display extension for the last three years, and we’re grateful to David for this successful project.

— James Brock, Tokyo 2021

<hr/>

Click to launch this notebook on Binder cloud service (takes a minute to launch): <a href="https://mybinder.org/v2/gh/jamesdbrock/learn-you-a-haskell-notebook/master?urlpath=lab/tree/ihaskell_examples/WidgetRevival.ipynb" target="_top"><!-- need _top target for Github nbviewer iframe --><img src="https://mybinder.org/badge_logo.svg" /></a>

[IHaskell on Github](https://github.com/gibiansky/IHaskell) | [This notebook on Github](https://github.com/jamesdbrock/learn-you-a-haskell-notebook/blob/master/notebook_extra/WidgetRevival.ipynb) | [*Learn You a Haskell for Great Good!* Jupyter adaptation](https://github.com/jamesdbrock/learn-you-a-haskell-notebook) 

<hr/>

## Demonstration combining Widgets with Charts

This is a demonstration of how to combine [*ihaskell-widgets*](https://github.com/gibiansky/IHaskell/tree/master/ihaskell-display/ihaskell-widgets) with [*ihaskell-charts*](https://github.com/gibiansky/IHaskell/tree/master/ihaskell-display/ihaskell-charts) for [*Chart*](https://hackage.haskell.org/package/Chart).

It renders an *ihaskell-widgets* `FloatRangeSlider` which allows you to select the beginning and end points of a numeric integral area under a function. The *ihaskell-charts* output renders the area under the function as a *Chart*.

See the [*Chart* wiki](https://github.com/timbod7/haskell-chart/wiki) for other possibilities.

In [1]:
import Graphics.Rendering.Chart
import IHaskell.Display.Widgets
import IHaskell.Display
import Data.Default.Class
import Control.Lens

f x = sin x * (x + 5.0)  -- The function which we will plot
domain = (0.0, 10.0)     -- The domain of the function which we will plot

grain = (snd domain - fst domain) / 100.0

integralTrapezoid ((x1,y1):(x2,y2):ps) = 
    ((x2 - x1) * ((y1 + y2) / 2.0)) + integralTrapezoid ((x2,y2):ps)
integralTrapezoid _ = 0

sliderRange <- mkFloatRangeSlider
setField sliderRange MinFloat (fst domain)
setField sliderRange MaxFloat (snd domain)
setField sliderRange StepFloat (Just grain)
setField sliderRange FloatPairValue (fst domain, snd domain)

outChart <- mkOutput

drawChart = do 
  (rangeMin, rangeMax) <- getField sliderRange FloatPairValue
  let 
      pts = [(x, f x) | x <- [fst domain, fst domain + grain .. snd domain]]    
      ptsRange = [(x,y) | (x,y) <- pts, x >= rangeMin, x <= rangeMax]
  displayCaption <- display $ plain 
      $ "Area under the curve = " <> show (integralTrapezoid ptsRange)
  displayChart <- display $ toRenderable $ def & layout_plots .~
      [ toPlot $ def & plot_lines_values .~ [pts]
      , toPlot $ def & plot_fillbetween_values .~ [(x,(0.0,y)) | (x,y) <- ptsRange]
      ]
  setField outChart Outputs [OutputData displayCaption, OutputData displayChart]
      
setField sliderRange ChangeHandler drawChart

sliderRange
outChart
drawChart

## Demonstration combining Widgets with Diagrams

This is a demonstration of how to combine [*ihaskell-widgets*](https://github.com/gibiansky/IHaskell/tree/master/ihaskell-display/ihaskell-widgets) with [*ihaskell-diagrams*](https://github.com/gibiansky/IHaskell/tree/master/ihaskell-display/ihaskell-diagrams) for [*diagrams*](https://hackage.haskell.org/package/diagrams).

It renders a `FloatSlider` widget and uses the widget value as one parameter of a rendering of the [__Apollonian gasket__](https://hackage.haskell.org/package/diagrams-contrib/docs/Diagrams-TwoD-Apollonian.html) from the [*diagrams-contrib*](https://hackage.haskell.org/package/diagrams-contrib) package.

See the [*Diagrams* project website](https://archives.haskell.org/projects.haskell.org/diagrams/) for other possibilities.

In [2]:
import Diagrams.Prelude
import Diagrams.TwoD.Apollonian
import IHaskell.Display.Widgets

sliderApollo <- mkFloatSlider
setField sliderApollo MinFloat 3.0
setField sliderApollo MaxFloat 9.0
setField sliderApollo StepFloat (Just 0.1)
setField sliderApollo FloatValue 6.0

outApollo <- mkOutput

drawApollo = do
  val <- getField sliderApollo FloatValue
  replaceOutput outApollo $ diagram $ apollonianGasket 0.01 2 4 val

setField sliderApollo ChangeHandler drawApollo

sliderApollo
outApollo
drawApollo